In [ ]:
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
import re
import numpy as np

## First scrape the main webpage

In [3]:
url = 'https://www.gov.za/speeches/statement-cabinet-meeting-wednesday-24-november-2021-25-nov-2021-0000'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'lxml')

## Article links follow a pattern  that starts with "news-details.php"
- This is used to compile a list of article links
- Sometimes the link is duplicated, this is then removed
- If this is done daily, one can just check for dulpicates again and removed since some artciles may feature on the main page for many days

In [3]:
pattern = r'news-details.php\?nid=\d+'
text= str(soup)
a_list=re.findall(pattern, text)
article_list= np.unique(np.array(a_list))

## Loop through the list
- Access each article and find the title and content
  - The title can be cleaned up more (it picks up "Welcome to Daily News etc.)
  - Content may also require further cleaning still, this is very raw
- No date yet (not so easy to find it, especially if format in html changes)

In [4]:
article_list

array(['news-details.php?nid=57449', 'news-details.php?nid=57453',
       'news-details.php?nid=57454', 'news-details.php?nid=57482',
       'news-details.php?nid=57483', 'news-details.php?nid=57484',
       'news-details.php?nid=57485', 'news-details.php?nid=57515',
       'news-details.php?nid=57516', 'news-details.php?nid=57545'],
      dtype='<U26')

In [2]:
data = pd.read_csv("../data/dailynews-gov-bw.csv")

In [3]:
data

,title,content,url
0,Welcome to Daily News .:. Dikgang - Puso e it...,Lephata la Botsogo le Boitekanelo le tlaa simo...,http://www.dailynews.gov.bw/news-details.php?n...
1,Welcome to Daily News .:. Dikgang - Tutume o ...,"Molaodi wa kgaolo potlana ya Tutume, Mme Utlwa...",http://www.dailynews.gov.bw/news-details.php?n...
2,Welcome to Daily News .:. Dikgang - Moimana k...,lick to see more pictures Batswana ba kile ba ...,http://www.dailynews.gov.bw/news-details.php?n...
3,Welcome to Daily News .:. Dikgang - Lephata l...,Mokwaledi wa lephata la Tlhabololo Temo-thuo l...,http://www.dailynews.gov.bw/news-details.php?n...
4,Welcome to Daily News .:. Dikgang - Dipetelel...,"Mogolwane wa mapodisi a Letlhakane, Superinten...",http://www.dailynews.gov.bw/news-details.php?n...
...,...,...,...
3616,Welcome to Daily News .:. Dikgang - Dikgosi k...,"Modulasetilo wa Ntlo ya Dikgosi,Kgosi Puso a r...",http://www.dailynews.gov.bw/news-details.php?n...
3617,Welcome to Daily News .:. Dikgang - Kgosi Pul...,"Mothusa kgosi ya Letlhakeng, Kgosi Tumelo Pule...",http://www.dailynews.gov.bw/news-details.php?n...
3618,Welcome to Daily News .:. Dikgang - Kgangyame...,Rre Tiroyaone Kgangyame a re tiro ya diatla e ...,http://www.dailynews.gov.bw/news-details.php?n...
3619,Welcome to Daily News .:. Dikgang - BoFiNet e...,Khamphani ya maranyane a ditlhaeletsanyo ya Bo...,http://www.dailynews.gov.bw/news-details.php?n...


In [3]:
data = pd.DataFrame(columns=['title','content','url'])

In [ ]:
hdr = {'User-Agent': 'Mozilla/5.0'} # Change depending on your browser or comment out, doesn't look like it affects this webpage
for page_num in range(811,900):
    print("Page :",page_num)
    offset = 10*(page_num-1) 
    url = 'http://www.dailynews.gov.bw/news-list.php?srccategory=10&page='+str(page_num)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'lxml')
    pattern = r'news-details.php\?nid=\d+'
    text= str(soup)
    a_list=re.findall(pattern, text)
    article_list= np.unique(np.array(a_list))
    for i in range(len(article_list)):
        url = 'http://www.dailynews.gov.bw/'+article_list[i]
        req = Request(url,headers=hdr)
        try:
            page = urlopen(req)
            soup = BeautifulSoup(page, 'lxml')
            text = soup.findAll('p',text = False) # Experiment with text = True (sometimes some content is not within the htmal tags: <p>...</p>)
            new = ''
            for t in text:
                new = new + str(t.find(text = True))
                new = new.replace("\r\n"," ")

            data.at[offset+i,'title'] = soup.title.text
            data.at[offset+i,'content'] = new[1:].replace('\t','').replace('\xa0','').replace('\n','').replace('Source:Author:Location:Event:Date:more  news »Main LinksNetworksSocial NetworkUseful LinksServicesContacts','')
            data.at[offset+i,'url'] = url
        except:
            pass
    data.to_csv("../data/dailynews-gov-bw.csv", index=None, header=True)
#data_csv = data.to_csv('DailyNewsBW.csv', index=None, header=True)

In [46]:
url = 'https://www.gov.za/speeches/statement-cabinet-meeting-wednesday-24-november-2021-25-nov-2021-0000'
req = Request(url,headers=hdr)
# try:
#     page = urlopen(req)
#     soup = BeautifulSoup(page, 'lxml')
#     text = soup.findAll('p',text = False) # Experiment with text = True (sometimes some content is not within the htmal tags: <p>...</p>)
#     new = ''
#     for t in text:
#         new = new + str(t.find(text = True))
#         new = new.replace("\r\n"," ")

#     data.at[offset+i,'title'] = soup.title.text
#     data.at[offset+i,'content'] = new[1:].replace('\t','').replace('\xa0','').replace('\n','').replace('Source:Author:Location:Event:Date:more  news »Main LinksNetworksSocial NetworkUseful LinksServicesContacts','')
#     data.at[offset+i,'url'] = url

In [64]:
page = urlopen(req)

In [65]:
soup = BeautifulSoup(page, 'lxml')

In [84]:
languages = ['af','nr','xh','zu','st','nso','ss','ve','ts last']

In [ ]:
news_item

In [88]:
len(soup.findAll('li',{'class':languages[0]}))

1

In [97]:
url = 'https://www.gov.za/cabinet-statements?page=1'
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'lxml')

/speeches/statement-virtual-cabinet-meeting-wednesday-10-march-2021-11-mar-2021-0000
/speeches/statement-virtual-cabinet-meeting-wednesday-26-feb-2021-0000
/speeches/statement-virtual-cabinet-meeting-10-february-2021-10-feb-2021-0000
/speeches/statement-virtual-special-cabinet-meeting-19-january-2021-19-jan-2021-0000
/speeches/special-cabinet-meeting-wednesday-9-december-10-dec-2020-0000
/speeches/statement-virtual-cabinet-meeting-wednesday-2-december-2020-4-dec-2020-0000
/speeches/statement-virtual-cabinet-meeting-18-november-2020-19-nov-2020-0000
/speeches/cabinet-meeting-wednesday-4-5-nov-2020-0000
/speeches/statement-virtual-cabinet-meeting-wednesday-21-october-2020-22-oct-2020-0000
/speeches/statement-virtual-cabinet-meeting-23-september-2020-24-sep-2020-0000
/speeches/statement-virtual-cabinet-meeting-9-september-2020-10-sep-2020-0000
/speeches/statement-virtual-cabinet-meeting-wednesday-26-august-2020-27-aug-2020-0000
/speeches/statement-virtual-cabinet-meeting-wednesday-5-augus

In [111]:
total_news = []

for i in range(10):
    url_page = 'https://www.gov.za/cabinet-statements?page=' + str(i)
    req_page = Request(url_page,headers=hdr)
    page_page = urlopen(req_page)
    soup_page = BeautifulSoup(page_page, 'lxml')
    
    for table_url in soup_page.findAll('td', {'class':'views-field views-field-title-field'}):
        slug = table_url.find('a')['href']
        print(slug)
        
        url = 'https://www.gov.za' + slug
        req = Request(url,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page, 'lxml')

        title = soup.findAll('h1',{'class':'title'})[0].text
        date = soup.findAll('span',{'class':'date-display-single'})[0]
        datetime_str = date['content']
        date_str = date.text
        text = soup.findAll('div',{'class':'field field-name-body field-type-text-with-summary field-label-hidden'})[0].text.replace('\xa0',' ')
        if len(soup.findAll('li',{'class':languages[0]})) ==1:
            news_item = {}
            news_item['title'] = title
            news_item['url'] = url
            news_item['date'] = date_str
            news_item['datetime'] = datetime_str
            news_item['en'] = {'text':text,'title':title, 'url':url}
            for lang in languages:
                url_trans = 'https://www.gov.za' + soup.findAll('li',{'class':lang})[0].findAll('a',{'class':'language-link'})[0]['href']
                req_trans = Request(url_trans,headers=hdr)
                page_trans = urlopen(req_trans)
                soup_trans = BeautifulSoup(page_trans, 'lxml')
                title_trans = soup_trans.findAll('h1',{'class':'title'})[0].text
                text_trans = soup_trans.findAll('div',{'class':'field field-name-body field-type-text-with-summary field-label-hidden'})[0].text.replace('\xa0',' ')
                news_item[lang] = {'text':text_trans, 'title':title_trans, 'url':url_trans}
            total_news.append(news_item)
            print("Saved: ", title)

/speeches/statement-cabinet-meeting-wednesday-9-march-2022-10-mar-2022-0000
/speeches/statement-cabinet-meeting-23-february-2022-23-feb-2022-0000
Saved:  Statement on the Cabinet Meeting of 23 February 2022
/speeches/statement-cabinet-meeting-9-february-2022-11-feb-2022-0000
Saved:  Statement of the virtual Cabinet Meeting of 9 February 2022
/speeches/statement-cabinet-lekgotla-cabinet-meeting-27-28-jan-2022
Saved:  Statement on the Cabinet Lekgotla and Special Cabinet Meeting of 27 and 28 January 2022
/speeches/statement-special-cabinet-meeting-thursday-30-december-2021-31-dec-2021-0000
/speeches/cabinet-approves-several-changes-adjusted-alert-level-1-covid-19-regulations-30-dec-2021
/speeches/statement-cabinet-meeting-08-december-2021-9-dec-2021-0000
Saved:  Statement on the Cabinet Meeting of 8 December 2021
/speeches/statement-cabinet-meeting-wednesday-24-november-2021-25-nov-2021-0000
Saved:  Statement on the Cabinet Meeting of 24 November 2021
/speeches/statement-virtual-cabinet-

In [112]:
len(total_news)

148

In [114]:
import json

In [115]:
final = json.dumps(total_news, indent=2)

In [116]:
with open("../data/govza-cabinet-statements.json", "w") as final:
   json.dump(total_news, final)